In [ ]:
import warnings 
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [ ]:
import os 
import tensorflow as tf 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,LearningRateScheduler

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPool2D,Input,InputLayer,Dropout,GlobalAveragePooling2D,Activation,Add
from tensorflow.keras.regularizers import L1,L2
from tensorflow.keras.metrics import CategoricalAccuracy,TopKCategoricalAccuracy

In [ ]:
CONFIGS = {
    "BATCH_SIZE":32,
    "IM_SIZE": 256,
    "N_FILTERS": 32,
    "LEARNING_RATE": 0.001,
    "N_EPOCHS":20,
    "DROPOUT_RATE": 0.0,
    "KERNEL_SIZE": 3,
    "N_FILTERS": 6,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_UNITS_1": 100,
    "N_DENSE_UNITS_2": 10,
    "N_CLASSES": 3,
    "REGULARIZER_RATE": 0.001,
    "LOSS": CategoricalCrossentropy(),
    "OPTIMIZER": Adam(learning_rate=0.001),
    "METRICS": ['accuracy']
}

In [ ]:
train_dir = r"EmotionsDatset\Emotions Dataset\Emotions Dataset\train"
val_dir = r"EmotionsDatset\Emotions Dataset\Emotions Dataset\test"
CLASS_NAME = ["angry","happy","sad"]


In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAME,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=99
) 

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=CLASS_NAME,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=99
) 

In [ ]:
for i in val_dataset.take(1):
    print(i)

In [ ]:
plt.figure(figsize=(12,12))
for image,label in train_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(image[i]/255.0)

        plt.title(CLASS_NAME[tf.argmax(label[i],axis = 0).numpy()])
        # plt.axis('off')

In [ ]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.layers import Resizing,Rescaling,Dense,Flatten

In [ ]:
resize_rescale_layer = Sequential([
    Resizing(CONFIGS['IM_SIZE'], CONFIGS['IM_SIZE']),
    Rescaling(1./255)
])

In [ ]:
lenet_model_builder = Sequential([
    InputLayer(input_shape = (CONFIGS['IM_SIZE'], CONFIGS['IM_SIZE'], 3)),
    Resizing(CONFIGS['IM_SIZE'], CONFIGS['IM_SIZE']),
    Rescaling(1./255),
    Conv2D(filters = CONFIGS['N_FILTERS'], kernel_size = CONFIGS['KERNEL_SIZE'],strides=CONFIGS['N_STRIDES'], activation = 'relu', padding = 'same', kernel_regularizer = L2(CONFIGS['REGULARIZER_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGS['POOL_SIZE'], strides=CONFIGS['N_STRIDES']*2),
    Dropout(rate=CONFIGS['DROPOUT_RATE']),
    Conv2D(filters = CONFIGS['N_FILTERS']*2+4, kernel_size = CONFIGS['KERNEL_SIZE'], strides=CONFIGS['N_STRIDES'], activation = 'relu', padding = 'same', kernel_regularizer = L2(CONFIGS['REGULARIZER_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGS['POOL_SIZE'], strides=CONFIGS['N_STRIDES']*2),
    Flatten(),
    
    Dense(units = CONFIGS['N_DENSE_UNITS_1'],activation='relu',kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE'])),

    BatchNormalization(),
    Dropout(rate=CONFIGS['DROPOUT_RATE']),

    Dense(units = CONFIGS['N_DENSE_UNITS_2'],activation='relu',kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE'])),

    BatchNormalization(),

    Dense(units = CONFIGS["N_CLASSES"], activation='softmax', kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE']))
])

In [ ]:
lenet_model_builder.summary()

In [ ]:
lenet_model_builder.compile(
    optimizer=CONFIGS['OPTIMIZER'], 
    loss=CONFIGS['LOSS'],
    metrics=[
        CategoricalAccuracy(name = 'accuracy'),
        TopKCategoricalAccuracy(k = 2,name = 'top-2-accuracy')
        ]
)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
lenet_model_builder.fit(
    train_dataset,validation_data = val_dataset,
    epochs = CONFIGS['N_EPOCHS'],
    verbose = 1,
    callbacks=[
        earlystop
    ]

)

In [ ]:
lenet_model_builder.evaluate(val_dataset)

In [ ]:
plt.plot(lenet_model_builder.history.history['accuracy'], label='train_accuracy')
plt.plot(lenet_model_builder.history.history['val_accuracy'], label='val_accuracy')
plt.plot(lenet_model_builder.history.history['val_loss'], label='val_loss')
plt.plot(lenet_model_builder.history.history['loss'], label='training_loss')

plt.xlabel('Epochs')   
plt.legend()

In [ ]:
plt.figure(figsize=(16,16))

for image, label in val_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4,4,i+1)
        plt.imshow(image[i] / 255.0)

        # True label
        true_class = CLASS_NAME[int(tf.argmax(label[i], axis=0))]

        # Model prediction (add batch dimension)
        preds = lenet_model_builder(tf.expand_dims(image[i], axis=0), training=False)
        pred_class = CLASS_NAME[int(tf.argmax(preds, axis=1))]

        plt.title(f"True Label: {true_class}\nPredicted Class: {pred_class}")
        plt.axis("off")

In [ ]:
predicted = []
labels = []

for image,label in val_dataset:
    predicted.append(lenet_model_builder(image,training = False)) 
    labels.append(label.numpy())  

In [ ]:
print(np.argmax(labels[:-1], axis=-1).flatten())
print(np.argmax(predicted[:-1], axis=-1).flatten())

In [ ]:
preds = np.argmax(predicted[:-1], axis=-1).flatten()
labels = np.argmax(labels[:-1], axis=-1).flatten()

In [ ]:
from sklearn.metrics import confusion_matrix

import seaborn as sns


In [ ]:
cm = confusion_matrix(labels, preds)
sns.heatmap(cm,annot=True, fmt='d', cmap='Blues', xticklabels=CLASS_NAME, yticklabels=CLASS_NAME)
plt.xlabel('Predicted')

In [ ]:
from tensorflow.keras.layers import RandomContrast,RandomFlip,RandomRotation

In [ ]:
augment_layers = Sequential([
    RandomFlip(mode = "horizontal"),
    RandomRotation(factor =(0.25,0.25)),
    RandomContrast(0.1),
])

def augument_data(image,label):
    return augment_layers(image,training = True),label

In [ ]:
train_dataset_aug = (
    train_dataset.map(augument_data,num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
)

In [ ]:
lenet_model_aug = Sequential([
    InputLayer(input_shape = (CONFIGS['IM_SIZE'], CONFIGS['IM_SIZE'], 3)),
    Resizing(CONFIGS['IM_SIZE'], CONFIGS['IM_SIZE']),
    Rescaling(1./255),
    Conv2D(filters = CONFIGS['N_FILTERS'], kernel_size = CONFIGS['KERNEL_SIZE'],strides=CONFIGS['N_STRIDES'], activation = 'relu', padding = 'same', kernel_regularizer = L2(CONFIGS['REGULARIZER_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGS['POOL_SIZE'], strides=CONFIGS['N_STRIDES']*2),
    Dropout(rate=CONFIGS['DROPOUT_RATE']),
    Conv2D(filters = CONFIGS['N_FILTERS']*2+4, kernel_size = CONFIGS['KERNEL_SIZE'], strides=CONFIGS['N_STRIDES'], activation = 'relu', padding = 'same', kernel_regularizer = L2(CONFIGS['REGULARIZER_RATE'])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGS['POOL_SIZE'], strides=CONFIGS['N_STRIDES']*2),
    Flatten(),
    
    Dense(units = CONFIGS['N_DENSE_UNITS_1'],activation='relu',kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE'])),

    BatchNormalization(),
    Dropout(rate=CONFIGS['DROPOUT_RATE']),

    Dense(units = CONFIGS['N_DENSE_UNITS_2'],activation='relu',kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE'])),

    BatchNormalization(),

    Dense(units = CONFIGS["N_CLASSES"], activation='softmax', kernel_regularizer=L2(CONFIGS['REGULARIZER_RATE']))
])
lenet_model_aug.compile(
    optimizer='adam', 
    loss=CONFIGS['LOSS'],
    metrics=[
        CategoricalAccuracy(name = 'accuracy'),
        TopKCategoricalAccuracy(k = 2,name = 'top-2-accuracy')
        ]
)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
lenet_model_aug.fit(
    train_dataset_aug,validation_data = val_dataset,
    epochs = CONFIGS['N_EPOCHS'],
    verbose = 1,
    callbacks=[
        earlystop
    ]
)

In [ ]:
lenet_model_aug.evaluate(val_dataset)

In [ ]:
import pickle

# Save
with open("LeNet_history.pkl", "wb") as f:
    pickle.dump(lenet_model_builder.history, f)